# Sonification of Temporally Triggered Sound Events
In numerous MIR applications one encounters cases where - for the time being not further specified - events occur at certain temporal positions. 
In this notebook, we illustrate how the `libsoni.core.tse` module can be used to sonify these events with different methods.

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os
import libfmp.b
import libfmp.c4
import librosa
from IPython import display as ipd

import libsoni

AUDIO_DIR = 'data_audio'
CSV_DIR = 'data_csv'
Fs = 22050

## Simple Scenario: Time Positions
To start with a simple example, let's create a list of arbitrarily chosen time positions, given in seconds:

In [ ]:
# Define some time positions
time_positions = [0.5, 1.25, 2.5, 2.75]

### Sonification of Time Positions with Clicks
Assuming we want to sonify this list, we can use the function `sonify_tse_clicks` to generate an audio signal comprising clicks at the corresponding time positions.

In [ ]:
# Sonification using libsoni
sonified_time_positions = libsoni.sonify_tse_clicks(time_positions=time_positions)

print('Sonified time positions:')
ipd.display(ipd.Audio(sonified_time_positions, rate=Fs))

The clicks generated within `sonify_tse_click` can be adjusted in order to match the respective use case with the following parameters:
 - `click_pitch`: pitch of the click
 - `click_reverb_duration`: duration of the click
 - `click_amplitude`: amplitude of the click
 
Let's say we'd rather have deeper sounding, longer clicks, the above sonification changes as follows:

In [ ]:
# Sonification using libsoni
sample_sonified_time_positions = libsoni.sonify_tse_clicks(time_positions=time_positions,
                                                           click_pitch=51,
                                                           click_fading_duration=0.5)

print('Sonified time positions with deeper, longer clicks:')
ipd.display(ipd.Audio(sample_sonified_time_positions, rate=Fs))

### Sonification of Time Positions with Samples
Before we get to a real-world audio example, let's stay with the arbitrarily chosen time positions and explain how samples can be used for sonification.
The first thing we need for this is - guess three times - a sample. Why don't we use a finger snap sound?

In [ ]:
# Load sample
snap_sample, _ = librosa.load(os.path.join(AUDIO_DIR,'samples', 'snap.wav'), sr=Fs)

print('Snap sample:')
ipd.display(ipd.Audio(snap_sample, rate=Fs))

Now, using the function `sonify_tse_sample` we can sonify our list of time positions with our sample:

In [ ]:
# Sonification using libsoni
sample_sonified_arbitrarily_chosen_time_positions = libsoni.sonify_tse_sample(time_positions=time_positions,
                                                                              sample=snap_sample)
                                                 
print('Sonified time positions with snap sample:')
ipd.display(ipd.Audio(sample_sonified_arbitrarily_chosen_time_positions, rate=Fs))

## Scenario 1: Sonifying Beat Annotations

When it comes to applications that deal with tempo and rhythm, one often encounters so-called beat annotations. In the following example we show how these are sonified. The associated audio examples are excerpts from:

 - String Quartet No. 2, 3rd movement by Alexander Borodin
 - Mazurka in F Major, Op. 68 by Frédéric Chopin
 - Piano Quartet No. 1 in C minor, Op. 15 by Gabriel Fauré

In [ ]:
# Borodin
title = 'Borodin: String Quartet No. 2, 3rd movement' 
fn_ann = os.path.join(CSV_DIR, 'demo_tse', 'FMP_C6_Audio_Borodin-sec39_RWC_quarter.csv')
fn_wav = os.path.join(AUDIO_DIR, 'demo_tse', 'FMP_C6_Audio_Borodin-sec39_RWC.wav')

libsoni.utils.plot_sonify_novelty_beats(fn_wav, fn_ann, title);

borodin_audio, _ = librosa.load(fn_wav, sr=Fs)
borodin_df = pd.read_csv(fn_ann)

# Sonification using libsoni
sonified_borodin = libsoni.sonify_tse_clicks(time_positions=borodin_df.to_numpy())

print('Original audio and sonification of beat positions with clicks:')
ipd.display(ipd.Audio(
    libsoni.utils.mix_sonification_and_original(sonified_borodin, borodin_audio, panning = 0.5), rate=Fs
))

# Chopin
title = 'Chopin: Op.68, No. 3' 
fn_ann = os.path.join(CSV_DIR, 'demo_tse', 'FMP_C6_Audio_Chopin.csv')
fn_wav = os.path.join(AUDIO_DIR, 'demo_tse', 'FMP_C6_Audio_Chopin.wav')

libsoni.utils.plot_sonify_novelty_beats(fn_wav, fn_ann, title);

chopin_audio, _ = librosa.load(fn_wav, sr=Fs)
chopin_df = pd.read_csv(fn_ann)

# Sonification using libsoni
castanets_sample, _ = librosa.load(os.path.join(AUDIO_DIR,'samples', 'castanets.wav'), sr=Fs)
sonified_chopin = libsoni.sonify_tse_sample(time_positions=chopin_df.to_numpy(),
                                            sample=castanets_sample)

print('Original audio and sonification of beat positions with castanet sample:')
ipd.display(ipd.Audio(
    libsoni.utils.mix_sonification_and_original(sonified_chopin, chopin_audio, panning = 0.5), rate=Fs
))


# Fauré
title = 'Fauré: Op.15' 
fn_ann = os.path.join(CSV_DIR, 'demo_tse', 'FMP_C6_Audio_Faure_Op015-01-sec0-12_SMD126.csv')
fn_wav = os.path.join(AUDIO_DIR, 'demo_tse', 'FMP_C6_Audio_Faure_Op015-01-sec0-12_SMD126.wav')

libsoni.utils.plot_sonify_novelty_beats(fn_wav, fn_ann, title);

faure_audio, _ = librosa.load(fn_wav, sr=Fs)
faure_df = pd.read_csv(fn_ann)

# Sonification using libsoni
metronome_sample, _ = librosa.load(os.path.join(AUDIO_DIR, 'samples', 'metronome.wav'), sr=Fs)
sonified_faure = libsoni.sonify_tse_sample(time_positions=faure_df.to_numpy(),sample=metronome_sample)

print('Original audio and sonification of beat positions with metronome sample:')
ipd.display(ipd.Audio(
    libsoni.utils.mix_sonification_and_original(sonified_faure, faure_audio, panning = 0.5), rate=Fs
))

## Scenario 2: Drumset - *Another One Bites The Dust* by *Queen*
The possibilities of the module are not limited to the use of one sample. Why don't we sonify drums with `sonify_tse_multiple_samples`. The underlying annotation to Queen's Another one bites the tust describes time positions with labels of the respective played drums or cymbals

In [ ]:
fn_ann = os.path.join(CSV_DIR, 'demo_tse', 'FMP_C6_F01_Queen_drums.csv')
fn_wav = os.path.join(AUDIO_DIR, 'demo_tse', 'FMP_C6_Audio_Queen_AnotherOneBitesTheDust-Beginning.wav')

queen_audio, _ = librosa.load(fn_wav, sr=Fs)
print('Another one bites the dust by Queen:')
ipd.display(ipd.Audio(queen_audio, rate=Fs))

queen_drums_df = pd.read_csv(fn_ann, delimiter =';')
print('Drums annotation:')
ipd.display(queen_drums_df)

The `sonify_tse_multiple_samples` function takes a list consisting of tuples of one array of time positions and one array of the corresponding sample. In the next cell we show what such a data structure looks like:

In [ ]:
kick_times = queen_drums_df[queen_drums_df['label']=='kick']['position'].to_numpy()
hihat_times = queen_drums_df[queen_drums_df['label']=='hihat']['position'].to_numpy()
snare_times = queen_drums_df[queen_drums_df['label']=='snare']['position'].to_numpy()

hihat_sample, _ = librosa.load(os.path.join(AUDIO_DIR, 'samples', 'hi-hat.wav'),sr=Fs)
snare_sample, _ = librosa.load(os.path.join(AUDIO_DIR, 'samples', 'snare.wav'),sr=Fs)
kick_sample, _ = librosa.load(os.path.join(AUDIO_DIR, 'samples', 'bass-drum.wav'),sr=Fs)

kick_tuple = (kick_times, kick_sample)
snare_tuple = (snare_times, snare_sample)
hihat_tuple = (hihat_times, hihat_sample)


queen_drums = [hihat_tuple, kick_tuple, snare_tuple]

### Sonification

In [ ]:
queen_sonified = libsoni.sonify_tse_multiple_samples(queen_drums)

print('Original audio:')
ipd.display(ipd.Audio(queen_audio,rate=Fs))

print('Sonified drums annotation with libsoni:')
ipd.display(ipd.Audio(queen_sonified,rate=Fs))

print('Original audio with sonification (stereo):')
ipd.display(ipd.Audio(libsoni.utils.mix_sonification_and_original(queen_audio, queen_sonified), rate=Fs))

## Scenario 3: Structure Annotations aux *Town Musicians of Bremen*



<div style="margin-top: 20px;">
    <div style="display: flex; align-items: center;">
        <div style="flex: 1; text-align: center;padding-right: 20px;">
            <img src="figures/demo_tse/town_musicians_of_bremen.png" alt="Image" style="width: 95%; max-width: 400px;"/>
        </div>
        <div style="flex: 1;">
            <p style="text-align: left-align; max-width: 800px;">
            The "Town Musicians of Bremen" is a famous German folk tale collected by the Brothers Grimm. It tells the story of four aging animals – a donkey, a dog, a cat, and a rooster – who, having outlived their usefulness to their owners, embark on a journey to Bremen to become musicians. In the next example, we have so-called structure annotations that identify individual passages of Frédéric Chopin's Op. 28 No. 11 in B major, also known as Prelude No. 11. For more details about the structure annotations, see the FMP Notebook <a href="https://www.audiolabs-erlangen.de/resources/MIR/FMP/C4/C4S1_MusicStructureGeneral.html">Music Structure Analysis: General Principles</a>. The Bremen Town Musicians and Frédéric Chopin may have little to do with each other, but nevertheless we use characteristic sound samples of the animals to mark the beginnings of different structural passages.
            </p>
        </div>


In [ ]:
fn_ann = os.path.join(CSV_DIR, 'demo_tse', 'FMP_C4_Audio_Chopin_Op028-11_003_20100611-SMD.csv')
chopin_df = pd.read_csv(fn_ann, delimiter=';')

ann, color_ann = libfmp.c4.read_structure_annotation(fn_ann)
ipd.display(chopin_df)

fn_wav = os.path.join(AUDIO_DIR, 'demo_tse', 'FMP_C4_Audio_Chopin_Op028-11_003_20100611-SMD.wav')
chopin_audio, _ = librosa.load(fn_wav, sr=Fs)

times_A = chopin_df[chopin_df['label'] == 'A']['start'].to_numpy()
times_B = chopin_df[chopin_df['label'] == 'B']['start'].to_numpy()
times_C = chopin_df[chopin_df['label'] == 'C']['start'].to_numpy()
times_D = chopin_df[chopin_df['label'] == 'D']['start'].to_numpy()

sample_A, _ = librosa.load(os.path.join(AUDIO_DIR, 'samples', 'rooster.wav'), sr=Fs)
sample_B, _ = librosa.load(os.path.join(AUDIO_DIR, 'samples', 'cat.wav'), sr=Fs)
sample_C, _ = librosa.load(os.path.join(AUDIO_DIR, 'samples', 'dog.wav'), sr=Fs)
sample_D, _ = librosa.load(os.path.join(AUDIO_DIR, 'samples', 'donkey.wav'), sr=Fs)

collection = [(times_A, sample_A),(times_B, sample_B),(times_C, sample_C),(times_D, sample_D)]

sonification_bremen = libsoni.sonify_tse_multiple_samples(collection, offset_relative=0.5)

color_ann = {'A': [1, 0, 0, 0.2],'B': [0, 1, 0, 0.2],  'C': [0, 0, 1, 0.2], 'D': [1, 1, 0, 0.2]}


fig, ax = libfmp.b.plot_segments(ann, colors=color_ann, figsize=(6, 1))
plt.xlabel('Time (frames)');
plt.show()

ipd.display(ipd.Audio(
    libsoni.utils.mix_sonification_and_original(sonification_bremen, chopin_audio, panning = 0.5), rate=Fs
))